In [ ]:
!pip install catboost

In [3]:
import numpy as np
import random
import os
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed=42
seed_everything(seed) # Seed 고정

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

### 데이터 전처리(공통)
모델 학습을 위해 데이터 전처리를 수행하는 과정입니다. 결측값들을 'NAN' 값으로 채워주고 **bounced** 변수의 값을 문자형으로 치환해 주었습니다. 또한 학습 과정에서 **bounced** 변수의 값이 **not_bounced**인 데이터들만 사용하도록 데이터를 걸러내는 작업을 했습니다.

In [5]:
train.fillna('NAN', inplace=True)
test.fillna('NAN', inplace=True)

train['bounced']=train['bounced'].replace({0:'not_bounced',1:'bounced'})
test['bounced']=test['bounced'].replace({0:'not_bounced',1:'bounced'})

train = train[train['bounced'] == 'not_bounced']

본격적인 데이터 전처리 과정에 앞서 **sessionID**와 **userID** 컬럼을 제거하였고 앙상블에 활용한 2가지 모델의 데이터 전처리 과정에 차이가 있기 때문에 똑같은 데이터셋을 각각 두 개씩 생성하였습니다.

In [6]:
drop_table = ['sessionID','userID']
train1 = train.drop(drop_table,axis=1)
train2 = train.drop(drop_table,axis=1)
test1 = test.drop(drop_table,axis=1)
test2 = test.drop(drop_table,axis=1)

### 데이터 전처리(모델1)
첫 번째 모델은 **transaction_revenue** 변수는 제거하고 남은 수치형 변수들에는 MinMaxScaling을 적용하였습니다.

In [7]:
train1 = train1.drop('transaction_revenue', axis=1)
test1 = test1.drop('transaction_revenue', axis=1)

features = ['quality','duration','transaction']

scaler = MinMaxScaler()
train1[features] = scaler.fit_transform(train1[features])
test1[features] = scaler.transform(test1[features])

In [11]:
categorical_features = [
    "browser",
    "OS",
    "device",
    "continent",
    "subcontinent",
    "country",
    "traffic_source",
    "traffic_medium",
    "keyword",
    "referral_path",
]
for i in categorical_features:
    train1[i] = train1[i].astype('category')
    test1[i] = test1[i].astype('category')

train1_X = train1.drop(['TARGET','bounced'], axis=1)
train1_y = train1['TARGET']

### 학습 및 예측(모델1)
모델은 CatBoostRegressor를 사용하였고 KFold 방식(k=5)을 적용하였습니다.
예측값은 각각의 모델에서 예측값을 출력하고 평균을 낸 값을 최종 예측값으로 사용하였습니다.

In [12]:
kf = KFold(n_splits=5)
models = []
for train_index, test_index in kf.split(train1_X):
    models.append(CatBoostRegressor(random_state=seed, verbose=False))
    kfold_train_X, kfold_train_y = train1_X.iloc[train_index], train1_y.iloc[train_index]
    kfold_test_X,  kfold_test_y  = train1_X.iloc[test_index],  train1_y.iloc[test_index]
    train_pool = Pool(data=kfold_train_X, label=kfold_train_y, cat_features=categorical_features)
    models[-1].fit(train_pool)

In [13]:
test_pool = Pool(data=test1.drop('bounced', axis=1), cat_features=categorical_features)
pred1 = np.array([models[0].predict(test_pool)])
for i in range(1, 5):
  pred1 = np.append(pred1, np.array([models[i].predict(test_pool)]),axis=0)
pred1 = np.mean(pred1, axis=0)

### 모델 예측값 후처리
모델에서 예측한 값들을 후처리하는 과정입니다.
Train 데이터를 살펴본 결과 Target 값은 1보다 크거나 같은 값만 존재한다고 판단하여 예측값이 1보다 작은 경우 1로 만들어주었습니다.
또한 Train 데이터에서 **bounced** 변수가 1인 경우 $($저는 0을 **not_bounced**로 1을 **bounced**로 치환했습니다.$)$ Target 값이 무조건 1로 고정되어 있었던 것을 보고 **bounced**가 1일 경우 예측값을 1로 후처리를 해주었습니다.

In [28]:
pred1 = [1 if i < 1 else i for i in pred1]
for i, bounced in enumerate(test.bounced):
  if bounced == 'bounced':
    pred1[i] = 1

### 데이터 전처리(모델2)

두 번째 모델은 **quality**, **duration** 변수들에 로그 변환을 적용한 후 모든 수치형 변수에 MinMaxScaling을 적용하였습니다. 추가로 **Target** 값에도 로그 변환을 적용하였습니다.


In [20]:
def log_transformation(data):
    features = ['quality','duration']
    for feature in features:
        data[feature] = np.log1p(data[feature])
    if 'TARGET' in data.columns:
        data['TARGET'] = np.log1p(data['TARGET'])
    return data

In [23]:
train2 = log_transformation(train2)
test2 = log_transformation(test2)

features = ['quality','duration','transaction','transaction_revenue']

scaler = MinMaxScaler()
train2[features] = scaler.fit_transform(train2[features])
test2[features] = scaler.transform(test2[features])

In [25]:
categorical_features = [
    "browser",
    "OS",
    "device",
    "continent",
    "subcontinent",
    "country",
    "traffic_source",
    "traffic_medium",
    "keyword",
    "referral_path",
]
for i in categorical_features:
    train2[i] = train2[i].astype('category')
    test2[i] = test2[i].astype('category')

train2_X = train2.drop(['TARGET','bounced'], axis=1)
train2_y = train2['TARGET']

### 학습 및 예측(모델2)
모델 1과 같이 CatBoostRegressor와 KFold 방식을 사용하였습니다. 예측값은 각각의 모델에서 예측값에 역로그 변환을 적용한 후 그 값들의 평균을 최종 예측값으로 사용하였습니다.

In [42]:
kf = KFold(n_splits=5)
models = []
for train_index, test_index in kf.split(train2_X):
    models.append(CatBoostRegressor(random_state=seed, verbose=False))
    kfold_train_X, kfold_train_y = train2_X.iloc[train_index], train2_y.iloc[train_index]
    kfold_test_X,  kfold_test_y  = train2_X.iloc[test_index],  train2_y.iloc[test_index]
    train_pool = Pool(data=kfold_train_X, label=kfold_train_y, cat_features=categorical_features)
    models[-1].fit(train_pool)

In [43]:
test_pool = Pool(data=test2.drop('bounced', axis=1), cat_features=categorical_features)
pred2 = np.array(np.expm1([models[0].predict(test_pool)]))
for i in range(1, 5):
  pred2 = np.append(pred2, np.expm1(np.array([models[i].predict(test_pool)])),axis=0)
pred2 = np.mean(pred2, axis=0)

### 모델 예측값 후처리
모델 1의 예측값 후처리 과정과 동일합니다.

In [44]:
pred2 = [1 if i < 1 else i for i in pred2]
for i, bounced in enumerate(test2.bounced):
  if bounced == 'bounced':
    pred2[i] = 1

### 최종 예측값 산출 및 출력
모델1과 모델2의 예측값의 평균을 구하고 제출 파일을 생성합니다.

In [47]:
pred = [(p1+p2)/2 for p1,p2 in zip(pred1, pred2)]

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit['TARGET'] = pred

In [ ]:
submit.to_csv("submission.csv", index=False)